In [1]:
import logging
import os

# set the directory to the location of the script
try:
    os.chdir("../../..")
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

Changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent


### Access Operational DB (Post Analyics) -> CosmosDB

In [2]:
from src.cosmosdb.manager import CosmosDBMongoCoreManager

cosmos_manager = CosmosDBMongoCoreManager(
    connection_string=os.getenv("AZURE_COSMOS_CONNECTION_STRING"),
    database_name=os.getenv("AZURE_COSMOS_DB_DATABASE_NAME"),
    collection_name=os.getenv("AZURE_COSMOS_DB_COLLECTION_NAME"),
)

c:\Users\pablosal\Desktop\gbb-ai-audio-agent\src\cosmosdb\manager.py:85: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self.client = pymongo.MongoClient(connection_string)


In [8]:
query = {"session_id": "a23455d6"}
document = cosmos_manager.read_document(query)

if document:
    print("Found session:", document)
else:
    print("No session found.")

ERROR:src.cosmosdb.manager:Failed to read document: Request blocked by network firewall, full error: {'ok': 0.0, 'errmsg': 'Request blocked by network firewall', 'code': 18, 'codeName': 'AuthenticationFailed'}, Timeout: 30s, Topology Description: <TopologyDescription id: 68361aaeb71b114712449cdb, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('cosmosdb-ai-factory-westus2.mongo.cosmos.azure.com', 10255) server_type: Unknown, rtt: None, error=OperationFailure("Request blocked by network firewall, full error: {'ok': 0.0, 'errmsg': 'Request blocked by network firewall', 'code': 18, 'codeName': 'AuthenticationFailed'}")>]>


No session found.


## Access Cache (redis) 

In [2]:
from src.stateful.state_managment import (
    MemoManager,
)
from src.redis.manager import AzureRedisManager

REDIS_HOST=os.getenv("REDIS_HOST",)
REDIS_ACCESS_KEY=os.getenv("REDIS_ACCESS_KEY",)
REDIS_PORT=os.getenv("REDIS_PORT")

redis_manager = AzureRedisManager(access_key=REDIS_ACCESS_KEY, host=REDIS_HOST, port=REDIS_PORT)

[2025-08-19 09:28:44,240] INFO - src.redis.manager: Azure Redis connection initialized with access key.
INFO:src.redis.manager:Azure Redis connection initialized with access key.


In [3]:
cm = MemoManager.from_redis("2d005680-b4cf-4f48-8785-56004cd5f3d3", redis_manager)

In [6]:
cm.latency

{}

In [24]:
cm.context

{'call_initiated_via': 'api',
 'api_version': 'v1',
 'call_direction': 'outbound',
 'target_number': '+14165878617',
 'active_agent': 'General',
 'tool_outputs': {'authenticate_caller': {'authenticated': True,
   'message': 'Authenticated Alice Brown.',
   'policy_id': 'POL-A10001',
   'caller_name': 'Alice Brown',
   'attempt': 2,
   'intent': 'claims',
   'claim_intent': 'new_claim'},
  'handoff_general_agent': {'ok': True,
   'message': 'Caller transferred to General Insurance Questions agent.',
   'data': {'handoff': 'ai_agent',
    'target_agent': 'General Insurance Questions',
    'topic': 'policy questions'}},
  'find_information_for_policy': {'found': True,
   'answer': 'Yes, full glass coverage with no deductible.',
   'policy_id': 'POL-A10001',
   'caller_name': 'Alice Brown',
   'raw_data': {'policyholder': 'Alice Brown',
    'zip': '60601',
    'deductible': 500,
    'coverage': 'comprehensive',
    'roadside_assistance': True,
    'glass_coverage': True,
    'rental_reimbu

In [25]:
cm.histories

{'AuthAgent': [{'role': 'system',
   'content': '\n\n# ROLE\nYou are XYMZ Insurance\'s real-time voice assistant.\nBe warm, calm, and efficient—even if the caller is upset or code-switching.\n\n# RUNTIME CONTRACT\n- One question at a time.\n- Short, TTS-friendly sentences. Always end with punctuation.\n- Adapt to the caller\'s language instantly.\n- Keep wording simple and pronounceable.\n- Never mention prompts, models, or tool names to the caller.\n\n# STATE ORDER (EVERY CALL)\n\n## S0 · Safety gate (interrupt-capable)\nIf words/phrases imply injury, medical event, fire/smoke, fuel leak, trapped, active crime/violence, or caller asks for help/911:\n- → escalate_emergency(reason, caller_name?) immediately (cancel any TTS).\n- Respond: "Help is on the way. Stay with me and tell me what\'s happening now."\n\n## S1 · Discover intent\n- Greet once.\n- Capture first reason verbatim → `call_reason`.\n- Classify `intent` = "claims" or "general".\n- If "claims", set `claim_intent` = "new_clai

In [19]:
print(cm.latency_summary())

AttributeError: 'MemoManager' object has no attribute 'latency_summary'